In [1]:
import geopandas as gpd
import geemap
import ee
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Load the GeoJSON file
geojson_path = 'JHB_GCRO_Selected_variables.geojson'
gdf = gpd.read_file(geojson_path)

# Sanitize the property names
def sanitize_column_names(gdf):
    gdf.columns = gdf.columns.str.replace('[^a-zA-Z0-9]', '_', regex=True)
    return gdf

gdf = sanitize_column_names(gdf)

# Convert the GeoDataFrame to an Earth Engine object
ee_fc = geemap.geopandas_to_ee(gdf)

# Define Area of Interest (AOI) from GeoJSON
aoi = ee_fc.geometry()

# Define date range
start_date = '2023-01-01'
end_date = '2023-12-31'

# Function to apply scale factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Function to mask clouds
def cloud_mask(image):
    cloud_shadow_bitmask = (1 << 3)
    cloud_bitmask = (1 << 5)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloud_shadow_bitmask).eq(0).And(qa.bitwiseAnd(cloud_bitmask).eq(0))
    return image.updateMask(mask)

# Load and process the Landsat 8 dataset
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
              .filterBounds(aoi) \
              .filterDate(start_date, end_date) \
              .map(apply_scale_factors) \
              .map(cloud_mask) \
              .median() \
              .clip(aoi)

# Visualization parameters
visualization = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 0.15}

# Create a map
Map = geemap.Map()

# Add layers to the map
Map.addLayer(landsat, visualization, 'True Color 432')

# Calculate NDVI
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
ndvi_palette = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
Map.addLayer(ndvi, ndvi_palette, 'NDVI Gauteng')

# Calculate LST
ndvi_min = ee.Number(ndvi.reduceRegion(ee.Reducer.min(), aoi, 30).values().get(0))
ndvi_max = ee.Number(ndvi.reduceRegion(ee.Reducer.max(), aoi, 30).values().get(0))

fv = ((ndvi.subtract(ndvi_min)).divide(ndvi_max.subtract(ndvi_min))).pow(ee.Number(2)).rename('FV')
em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')
thermal = landsat.select('ST_B10').rename('thermal')

lst = thermal.expression(
  '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {
    'TB': thermal.select('thermal'),
    'em': em
  }).rename('LST Gauteng 2023')

lst_palette = {
  'min': 18.47, 'max': 42.86, 'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ]
}

Map.addLayer(lst, lst_palette, 'Land Surface Temperature 2023')

# Add buildings dataset
buildings = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons").filterBounds(aoi)
Map.addLayer(buildings, {}, 'Buildings')

# Add socio-economic datasets
population_density = ee.ImageCollection("CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics") \
                          .filterDate('2020-01-01', '2020-12-31') \
                          .mean() \
                          .clip(aoi)

population_density_palette = {'min': 0, 'max': 1000, 'palette': ['blue', 'green', 'red']}
Map.addLayer(population_density, population_density_palette, 'Population Density')

night_lights = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG") \
                    .filterDate('2023-01-01', '2023-12-31') \
                    .select('avg_rad') \
                    .mean() \
                    .clip(aoi)

night_lights_palette = {'min': 0, 'max': 60, 'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']}
Map.addLayer(night_lights, night_lights_palette, 'Nighttime Lights')

# Display the map
Map.centerObject(ee_fc, 10)
Map



Successfully saved authorization token.


Map(center=[-26.176814917998072, 27.9632492022089], controls=(WidgetControl(options=['position', 'transparent_…

In [2]:
import geopandas as gpd
import geemap
import ee
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Load the GeoJSON file
geojson_path = 'JHB_GCRO_Selected_variables.geojson'
gdf = gpd.read_file(geojson_path)

# Sanitize the property names
def sanitize_column_names(gdf):
    gdf.columns = gdf.columns.str.replace('[^a-zA-Z0-9]', '_', regex=True)
    return gdf

gdf = sanitize_column_names(gdf)

# Select relevant columns for HVI calculation
columns = [
    'UTFVI', 'LST', 'NDVI', 'NDBI__mean', 'concern_he', 'cancer_pro',
    'diabetes_p', 'pneumonia_', 'heart_dise', 'hypertensi', 'hiv_prop',
    'tb_prop', 'covid_prop', '60_plus_pr'
]

# Ensure columns are numeric and fill missing values
for col in columns:
    gdf[col] = pd.to_numeric(gdf[col], errors='coerce')
gdf = gdf.dropna(subset=columns)

# Normalize the data
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(gdf[columns])

# Create the composite HVI by averaging the normalized features
gdf['HVI'] = normalized_features.mean(axis=1)

# Convert the GeoDataFrame to an Earth Engine object
ee_fc = geemap.geopandas_to_ee(gdf)

# Define Area of Interest (AOI) from GeoJSON
aoi = ee_fc.geometry()

# Define date range
start_date = '2023-01-01'
end_date = '2023-12-31'

# Function to apply scale factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Function to mask clouds
def cloud_mask(image):
    cloud_shadow_bitmask = (1 << 3)
    cloud_bitmask = (1 << 5)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloud_shadow_bitmask).eq(0).And(qa.bitwiseAnd(cloud_bitmask).eq(0))
    return image.updateMask(mask)

# Load and process the Landsat 8 dataset
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
              .filterBounds(aoi) \
              .filterDate(start_date, end_date) \
              .map(apply_scale_factors) \
              .map(cloud_mask) \
              .median() \
              .clip(aoi)

# Visualization parameters
visualization = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 0.15}

# Create a map
Map = geemap.Map()

# Add layers to the map
Map.addLayer(landsat, visualization, 'True Color 432')

# Calculate NDVI
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
ndvi_palette = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
Map.addLayer(ndvi, ndvi_palette, 'NDVI Gauteng')

# Calculate LST
ndvi_min = ee.Number(ndvi.reduceRegion(ee.Reducer.min(), aoi, 30).values().get(0))
ndvi_max = ee.Number(ndvi.reduceRegion(ee.Reducer.max(), aoi, 30).values().get(0))

fv = ((ndvi.subtract(ndvi_min)).divide(ndvi_max.subtract(ndvi_min))).pow(ee.Number(2)).rename('FV')
em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')
thermal = landsat.select('ST_B10').rename('thermal')

lst = thermal.expression(
  '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {
    'TB': thermal.select('thermal'),
    'em': em
  }).rename('LST Gauteng 2023')

lst_palette = {
  'min': 18.47, 'max': 42.86, 'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ]
}

Map.addLayer(lst, lst_palette, 'Land Surface Temperature 2023')

# Add buildings dataset
buildings = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons").filterBounds(aoi)
Map.addLayer(buildings, {}, 'Buildings')

# Add socio-economic datasets
population_density = ee.ImageCollection("CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics") \
                          .filterDate('2020-01-01', '2020-12-31') \
                          .mean() \
                          .clip(aoi)

population_density_palette = {'min': 0, 'max': 1000, 'palette': ['blue', 'green', 'red']}
Map.addLayer(population_density, population_density_palette, 'Population Density')

night_lights = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG") \
                    .filterDate('2023-01-01', '2023-12-31') \
                    .select('avg_rad') \
                    .mean() \
                    .clip(aoi)

night_lights_palette = {'min': 0, 'max': 60, 'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']}
Map.addLayer(night_lights, night_lights_palette, 'Nighttime Lights')

# Define visualization parameters for HVI
vis_params_hvi = {
    'min': gdf['HVI'].min(),
    'max': gdf['HVI'].max(),
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Add HVI layer to the map
ee_image_hvi = ee_fc.reduceToImage(['HVI'], ee.Reducer.first())
Map.addLayer(ee_image_hvi, vis_params_hvi, 'Heat Vulnerability Index (HVI)')

# Highlight ward boundaries
Map.addLayer(ee_fc, {}, 'Ward Boundaries')

# Center the map on the data
Map.centerObject(ee_fc, 10)

# Display the map
Map


Map(center=[-26.176814917998072, 27.9632492022089], controls=(WidgetControl(options=['position', 'transparent_…

In [3]:
import geopandas as gpd
import geemap
import ee
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Load the GeoJSON file
geojson_path = 'JHB_GCRO_Selected_variables.geojson'
gdf = gpd.read_file(geojson_path)

# Sanitize the property names
def sanitize_column_names(gdf):
    gdf.columns = gdf.columns.str.replace('[^a-zA-Z0-9]', '_', regex=True)
    return gdf

gdf = sanitize_column_names(gdf)

# Convert the GeoDataFrame to an Earth Engine object
ee_fc = geemap.geopandas_to_ee(gdf)

# Define Area of Interest (AOI) from GeoJSON
aoi = ee_fc.geometry()

# Define date range
start_date = '2023-01-01'
end_date = '2023-12-31'

# Function to apply scale factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Function to mask clouds
def cloud_mask(image):
    cloud_shadow_bitmask = (1 << 3)
    cloud_bitmask = (1 << 5)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloud_shadow_bitmask).eq(0).And(qa.bitwiseAnd(cloud_bitmask).eq(0))
    return image.updateMask(mask)

# Load and process the Landsat 8 dataset
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
              .filterBounds(aoi) \
              .filterDate(start_date, end_date) \
              .map(apply_scale_factors) \
              .map(cloud_mask) \
              .median() \
              .clip(aoi)

# Visualization parameters
visualization = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 0.15}

# Create a map
Map = geemap.Map()

# Add layers to the map
Map.addLayer(landsat, visualization, 'True Color 432')

# Calculate NDVI
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
ndvi_palette = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
Map.addLayer(ndvi, ndvi_palette, 'NDVI Gauteng')

# Calculate LST
ndvi_min = ee.Number(ndvi.reduceRegion(ee.Reducer.min(), aoi, 30).values().get(0))
ndvi_max = ee.Number(ndvi.reduceRegion(ee.Reducer.max(), aoi, 30).values().get(0))

fv = ((ndvi.subtract(ndvi_min)).divide(ndvi_max.subtract(ndvi_min))).pow(ee.Number(2)).rename('FV')
em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')
thermal = landsat.select('ST_B10').rename('thermal')

lst = thermal.expression(
  '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {
    'TB': thermal.select('thermal'),
    'em': em
  }).rename('LST Gauteng 2023')

lst_palette = {
  'min': 18.47, 'max': 42.86, 'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ]
}

Map.addLayer(lst, lst_palette, 'Land Surface Temperature 2023')

# Add buildings dataset
buildings = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons").filterBounds(aoi)
Map.addLayer(buildings, {}, 'Buildings')

# Add socio-economic datasets
population_density = ee.ImageCollection("CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics") \
                          .filterDate('2020-01-01', '2020-12-31') \
                          .mean() \
                          .clip(aoi)

population_density_palette = {'min': 0, 'max': 1000, 'palette': ['blue', 'green', 'red']}
Map.addLayer(population_density, population_density_palette, 'Population Density')

night_lights = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG") \
                    .filterDate('2023-01-01', '2023-12-31') \
                    .select('avg_rad') \
                    .mean() \
                    .clip(aoi)

night_lights_palette = {'min': 0, 'max': 60, 'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']}
Map.addLayer(night_lights, night_lights_palette, 'Nighttime Lights')

# Select relevant columns for HVI calculation
columns = [
    'UTFVI', 'LST', 'NDVI', 'NDBI__mean', 'concern_he', 'cancer_pro',
    'diabetes_p', 'pneumonia_', 'heart_dise', 'hypertensi', 'hiv_prop',
    'tb_prop', 'covid_prop', '60_plus_pr'
]

# Ensure columns are numeric and fill missing values
for col in columns:
    gdf[col] = pd.to_numeric(gdf[col], errors='coerce')
gdf = gdf.dropna(subset=columns)

# Normalize the data
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(gdf[columns])

# Create the composite HVI by averaging the normalized features
gdf['HVI'] = normalized_features.mean(axis=1)

# Convert the GeoDataFrame to an Earth Engine object
ee_fc = geemap.geopandas_to_ee(gdf)

# Define visualization parameters for HVI
vis_params_hvi = {
    'min': gdf['HVI'].min(),
    'max': gdf['HVI'].max(),
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Add HVI layer to the map
ee_image_hvi = ee_fc.reduceToImage(['HVI'], ee.Reducer.first())
Map.addLayer(ee_image_hvi, vis_params_hvi, 'Heat Vulnerability Index (HVI)')

# Highlight ward boundaries
Map.addLayer(ee_fc, {}, 'Ward Boundaries')

# Center the map on the data
Map.centerObject(ee_fc, 10)

# Display the map
Map


Map(center=[-26.176814917998072, 27.9632492022089], controls=(WidgetControl(options=['position', 'transparent_…

In [4]:
import geopandas as gpd
import geemap
import ee
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Load the GeoJSON file
geojson_path = 'JHB_GCRO_Selected_variables.geojson'
gdf = gpd.read_file(geojson_path)

# Sanitize the property names
def sanitize_column_names(gdf):
    gdf.columns = gdf.columns.str.replace('[^a-zA-Z0-9]', '_', regex=True)
    return gdf

gdf = sanitize_column_names(gdf)

# Convert the GeoDataFrame to an Earth Engine object
ee_fc = geemap.geopandas_to_ee(gdf)

# Define Area of Interest (AOI) from GeoJSON
aoi = ee_fc.geometry()

# Define date range
start_date = '2023-01-01'
end_date = '2023-12-31'

# Function to apply scale factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Function to mask clouds
def cloud_mask(image):
    cloud_shadow_bitmask = (1 << 3)
    cloud_bitmask = (1 << 5)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloud_shadow_bitmask).eq(0).And(qa.bitwiseAnd(cloud_bitmask).eq(0))
    return image.updateMask(mask)

# Load and process the Landsat 8 dataset
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
              .filterBounds(aoi) \
              .filterDate(start_date, end_date) \
              .map(apply_scale_factors) \
              .map(cloud_mask) \
              .median() \
              .clip(aoi)

# Visualization parameters
visualization = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 0.15}

# Create a map
Map = geemap.Map()

# Add layers to the map
Map.addLayer(landsat, visualization, 'True Color 432')

# Calculate NDVI
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
ndvi_palette = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
Map.addLayer(ndvi, ndvi_palette, 'NDVI Gauteng')

# Calculate LST
ndvi_min = ee.Number(ndvi.reduceRegion(ee.Reducer.min(), aoi, 30).values().get(0))
ndvi_max = ee.Number(ndvi.reduceRegion(ee.Reducer.max(), aoi, 30).values().get(0))

fv = ((ndvi.subtract(ndvi_min)).divide(ndvi_max.subtract(ndvi_min))).pow(ee.Number(2)).rename('FV')
em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')
thermal = landsat.select('ST_B10').rename('thermal')

lst = thermal.expression(
  '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {
    'TB': thermal.select('thermal'),
    'em': em
  }).rename('LST Gauteng 2023')

lst_palette = {
  'min': 18.47, 'max': 42.86, 'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ]
}

Map.addLayer(lst, lst_palette, 'Land Surface Temperature 2023')

# Add GHSL population data
population1975 = ee.Image('JRC/GHSL/P2023A/GHS_POP/1975').clip(aoi)
population1990 = ee.Image('JRC/GHSL/P2023A/GHS_POP/1990').clip(aoi)
population2020 = ee.Image('JRC/GHSL/P2023A/GHS_POP/2020').clip(aoi)

populationCountVis = {
  'min': 0.0,
  'max': 100.0,
  'palette': ['000004', '320A5A', '781B6C', 'BB3654', 'EC6824', 'FBB41A', 'FCFFA4']
}

population1975 = population1975.updateMask(population1975.gt(0))
population1990 = population1990.updateMask(population1990.gt(0))
population2020 = population2020.updateMask(population2020.gt(0))

Map.addLayer(population1975, populationCountVis, 'Population count, 1975')
Map.addLayer(population1990, populationCountVis, 'Population count, 1990')
Map.addLayer(population2020, populationCountVis, 'Population count, 2020')

# Add buildings dataset
buildings = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons").filterBounds(aoi)
Map.addLayer(buildings, {}, 'Buildings')

# Add socio-economic datasets
population_density = ee.ImageCollection("CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics") \
                          .filterDate('2020-01-01', '2020-12-31') \
                          .mean() \
                          .clip(aoi)

population_density_palette = {'min': 0, 'max': 1000, 'palette': ['blue', 'green', 'red']}
Map.addLayer(population_density, population_density_palette, 'Population Density')

night_lights = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG") \
                    .filterDate('2023-01-01', '2023-12-31') \
                    .select('avg_rad') \
                    .mean() \
                    .clip(aoi)

night_lights_palette = {'min': 0, 'max': 60, 'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']}
Map.addLayer(night_lights, night_lights_palette, 'Nighttime Lights')

# Select relevant columns for HVI calculation
columns = [
    'UTFVI', 'LST', 'NDVI', 'NDBI__mean', 'concern_he', 'cancer_pro',
    'diabetes_p', 'pneumonia_', 'heart_dise', 'hypertensi', 'hiv_prop',
    'tb_prop', 'covid_prop', '60_plus_pr'
]

# Ensure columns are numeric and fill missing values
for col in columns:
    gdf[col] = pd.to_numeric(gdf[col], errors='coerce')
gdf = gdf.dropna(subset=columns)

# Normalize the data
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(gdf[columns])

# Create the composite HVI by averaging the normalized features
gdf['HVI'] = normalized_features.mean(axis=1)

# Convert the GeoDataFrame to an Earth Engine object
ee_fc = geemap.geopandas_to_ee(gdf)

# Define visualization parameters for HVI
vis_params_hvi = {
    'min': gdf['HVI'].min(),
    'max': gdf['HVI'].max(),
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Add HVI layer to the map
ee_image_hvi = ee_fc.reduceToImage(['HVI'], ee.Reducer.first())
Map.addLayer(ee_image_hvi, vis_params_hvi, 'Heat Vulnerability Index (HVI)')

# Highlight ward boundaries
Map.addLayer(ee_fc, {}, 'Ward Boundaries')

# Center the map on the data
Map.centerObject(ee_fc, 10)

# Display the map
Map


Map(center=[-26.176814917998072, 27.9632492022089], controls=(WidgetControl(options=['position', 'transparent_…